In [36]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
import os

In [37]:
data_dir = "Train"
test_data_dir = "Test"

# Define data augmentation and normalization transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.RandomAffine(degrees=0, shear=30),  # Add shear transformation
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Add color jitter
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the train and test data with transformations
dataset = ImageFolder(data_dir, transform=train_transform)
test_dataset = ImageFolder(test_data_dir, transform=test_transform)

# Print some information about the loaded data
train_dogs = []
pic_count = 0
for folder in dataset.classes:
    files = os.listdir(os.path.join(data_dir, folder))
    train_dogs.append({"Folder": folder, "Count": len(files)})
    pic_count += len(files)

print("Number of train data:", pic_count)
train_dogs

Number of train data: 944


[{'Folder': 'Border Collie', 'Count': 112},
 {'Folder': 'Borzoi', 'Count': 110},
 {'Folder': 'Cocker', 'Count': 130},
 {'Folder': 'German Sheperd', 'Count': 109},
 {'Folder': 'Golden Retriever', 'Count': 127},
 {'Folder': 'Greyhound', 'Count': 109},
 {'Folder': 'Pomeranian', 'Count': 149},
 {'Folder': 'Shiba Inu', 'Count': 98}]

In [38]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

batch_size = 32
val_size = 100
train_size = len(dataset) - val_size 

train_data,val_data = random_split(dataset,[train_size,val_size])
print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Validation Data : {len(val_data)}")

Length of Train Data : 844
Length of Validation Data : 100


In [39]:
#load the train and validation into batches.
train_dl = DataLoader(train_data, batch_size, shuffle = True, num_workers = 4, pin_memory = True)
val_dl = DataLoader(val_data, batch_size*2, num_workers = 4, pin_memory = True)

In [40]:
import torch.nn as nn
import torch.nn.functional as F

In [41]:
class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs] # collect losses for batches
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]    # collect accuracies for batches
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [42]:
class DogClassification(ImageClassificationBase):
    def __init__(self, num_classes = 8):
        super(DogClassification, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fc = nn.Linear(200704,1024)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(1024, 512)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512,num_classes)
        # TODO softmax as activation for output?
        #  ∘ LeNet-5
    
        # resources to read up on architectures:
        # https://medium.com/@siddheshb008/lenet-5-architecture-explained-3b559cb2d52b
        # https://blog.paperspace.com/writing-lenet5-from-scratch-in-python/
        
        # more general resources
        # https://medium.com/@siddheshb008/why-convolutions-dd7641d2bf81
        # https://medium.com/@siddheshb008/understanding-convolution-neural-networks-a30211e12a06
        # https://medium.com/@siddheshb008/understanding-convolutional-neural-networks-part-2-98694dd47923
    
    def forward(self, xb):
        out = self.layer1(xb)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [43]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

  
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history

In [44]:
num_epochs = 10
opt_func = torch.optim.Adam
lr = 0.0001
model = DogClassification() # number of breeds

#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

Epoch [0], train_loss: 2.5884, val_loss: 2.0841, val_acc: 0.2543
Epoch [1], train_loss: 1.9327, val_loss: 1.8277, val_acc: 0.3385
Epoch [2], train_loss: 1.8246, val_loss: 1.7780, val_acc: 0.3368
Epoch [3], train_loss: 1.6459, val_loss: 1.6056, val_acc: 0.3767
Epoch [4], train_loss: 1.5402, val_loss: 1.7009, val_acc: 0.3724
Epoch [5], train_loss: 1.5926, val_loss: 1.5901, val_acc: 0.4253
Epoch [6], train_loss: 1.6210, val_loss: 1.7061, val_acc: 0.3724
Epoch [7], train_loss: 1.5381, val_loss: 1.5526, val_acc: 0.4470
Epoch [8], train_loss: 1.4371, val_loss: 1.6476, val_acc: 0.3524
Epoch [9], train_loss: 1.4785, val_loss: 1.5294, val_acc: 0.4288


In [45]:
opt_func2 = torch.optim.AdamW

In [46]:
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func2)

Epoch [0], train_loss: 1.5672, val_loss: 1.6020, val_acc: 0.3759
Epoch [1], train_loss: 1.4079, val_loss: 1.6707, val_acc: 0.4132
Epoch [2], train_loss: 1.4882, val_loss: 1.6366, val_acc: 0.3698
Epoch [3], train_loss: 1.4379, val_loss: 1.5122, val_acc: 0.4175
Epoch [4], train_loss: 1.3950, val_loss: 1.5292, val_acc: 0.4748
Epoch [5], train_loss: 1.3084, val_loss: 1.5997, val_acc: 0.4115
Epoch [6], train_loss: 1.3100, val_loss: 1.6297, val_acc: 0.4349
Epoch [7], train_loss: 1.3070, val_loss: 1.4867, val_acc: 0.4132
Epoch [8], train_loss: 1.3234, val_loss: 1.4418, val_acc: 0.5139
Epoch [9], train_loss: 1.2771, val_loss: 1.4277, val_acc: 0.4861
